In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pwd
!ls -R

/kaggle/working
.:


In [2]:
!git clone https://github.com/modelscope/ms-swift.git
%cd ms-swift

Cloning into 'ms-swift'...
remote: Enumerating objects: 22220, done.
remote: Counting objects: 100% (1827/1827), done.
remote: Compressing objects: 100% (738/738), done.
remote: Total 22220 (delta 1333), reused 1492 (delta 1053), pack-reused 20393 (from 1)
Receiving objects: 100% (22220/22220), 49.23 MiB | 40.17 MiB/s, done.
Resolving deltas: 100% (15890/15890), done.
/kaggle/working/ms-swift


In [3]:
!pip install -e .[llm]

Obtaining file:///kaggle/working/ms-swift
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.1/298.1 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.5/449.5 kB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 77.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 94.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.6/894.6 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [4]:
%cd ..

/kaggle/working


In [5]:
import os

# Set environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Kaggle typically provides a single GPU


In [6]:
! pip install verovio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 63.3 MB/s eta 0:00:0000:0100:01


In [8]:
import pandas as pd
import json

# Load the CSV data
file_path = '/kaggle/input/hindi-ocr-synthetic-line-image-text-pair/data_80k/data.csv'
df = pd.read_csv(file_path, encoding='utf-8')


In [9]:
import os
import json

# Base path for images
image_base_path = '/kaggle/input/hindi-ocr-synthetic-line-image-text-pair/data_80k/output_images/'

json_data = []

# Loop through the CSV data and create JSON objects
for index, row in df.iterrows():
    # Construct the full image path by combining the base path with the filename
    full_image_path = os.path.join(image_base_path, row['image_file'])
    
    json_obj = {
        "query": "<image>Transcribe the text in this image",  # Fixed query text
        "response": row['text'],                             # Expected output from the CSV
        "images": [full_image_path]                          # Full image path
    }
    json_data.append(json_obj)

# Convert the list of dictionaries to JSON format
json_output = json.dumps(json_data, indent=4, ensure_ascii=False)

# Save to a JSON file (optional)
output_file_path = '/kaggle/working/output_data.json'
with open(output_file_path, 'w', encoding='utf-8') as f:
    f.write(json_output)

print("JSON file created successfully!")


JSON file created successfully!


In [2]:
!swift sft \
--model_type got-ocr2 \
--model_id_or_path stepfun-ai/GOT-OCR2_0 \
--sft_type lora \
--dataset /kaggle/working/output_data.json \
--output_dir /kaggle/working/hindi_got_model_3 \
--num_train_epochs 1 \
--max_steps 1000 \
--per_device_train_batch_size 1 \
--gradient_accumulation_steps 4 \
--learning_rate 2e-5 \
--lora_rank 8 \
--lora_alpha 32 \
--lora_dropout 0.05 \
--evaluation_strategy steps \
--eval_steps 200 \
--save_strategy steps \
--save_steps 200

/bin/bash: line 1: swift: command not found


In [1]:
import os
import json

# Base path for images
image_base_path = '/kaggle/input/hindi-ocr-synthetic-line-image-text-pair/data_80k/TestSamples/'

json_data = []

# Loop through the CSV data and create JSON objects (limit to 10 files)
for index, row in df.iterrows():
    if index >= 10:
        break  # Stop after processing 10 files
    
    # Construct the full image path by combining the base path with the filename
    full_image_path = os.path.join(image_base_path, row['image_file'])
    
    json_obj = {
        "query": "<image>Transcribe the text in this image",  # Fixed query text
        "response": row['text'],                             # Expected output from the CSV
        "images": [full_image_path]                          # Full image path
    }
    json_data.append(json_obj)

# Convert the list of dictionaries to JSON format
json_output = json.dumps(json_data, indent=4, ensure_ascii=False)

# Save to a JSON file (optional)
output_file_path = '/kaggle/working/test1.json'
with open(output_file_path, 'w', encoding='utf-8') as f:
    f.write(json_output)

print("JSON file created successfully!")


NameError: name 'df' is not defined

In [13]:
!CUDA_VISIBLE_DEVICES=0 swift infer\
--ckpt_dir /kaggle/working/hindi_got_model_3/got-ocr2/v0-20240930-060444/checkpoint-1000 \
--dataset /kaggle/working/test1.json \
--load_dataset_config true 

run sh: `/opt/conda/bin/python3.10 /kaggle/working/ms-swift/swift/cli/infer.py --ckpt_dir /kaggle/working/hindi_got_model_3/got-ocr2/v0-20240930-060444/checkpoint-1000 --dataset /kaggle/working/test1.json --load_dataset_config true`
[INFO:swift] Successfully registered `/kaggle/working/ms-swift/swift/llm/data/dataset_info.json`
[INFO:swift] No vLLM installed, if you are using vLLM, you will get `ImportError: cannot import name 'get_vllm_engine' from 'swift.llm'`
[INFO:swift] No LMDeploy installed, if you are using LMDeploy, you will get `ImportError: cannot import name 'prepare_lmdeploy_engine_template' from 'swift.llm'`
[INFO:swift] Start time of running main: 2024-09-30 08:19:06.220834
[INFO:swift] ckpt_dir: /kaggle/working/hindi_got_model_3/got-ocr2/v0-20240930-060444/checkpoint-1000
[INFO:swift] Setting model_info['revision']: master
[INFO:swift] Setting args.eval_human: False
[INFO:swift] args: InferArguments(model_type='got-ocr2', model_id_or_path='stepfun-ai/GOT-OCR2_0', model_r

In [14]:
import os

# Define the path to the checkpoint
checkpoint_path = "/kaggle/working/hindi_got_model_3/got-ocr2/v0-20240930-060444/checkpoint-1000"

# Create a zip file name
zip_name = "checkpoint-1000.zip"

# Create the zip file
!zip -r /kaggle/working/{zip_name} {checkpoint_path}

print(f"Zip file created: /kaggle/working/{zip_name}")

  adding: kaggle/working/hindi_got_model_3/got-ocr2/v0-20240930-060444/checkpoint-1000/ (stored 0%)
  adding: kaggle/working/hindi_got_model_3/got-ocr2/v0-20240930-060444/checkpoint-1000/adapter_config.json (deflated 47%)
  adding: kaggle/working/hindi_got_model_3/got-ocr2/v0-20240930-060444/checkpoint-1000/training_args.bin (deflated 53%)
  adding: kaggle/working/hindi_got_model_3/got-ocr2/v0-20240930-060444/checkpoint-1000/infer_result/ (stored 0%)
  adding: kaggle/working/hindi_got_model_3/got-ocr2/v0-20240930-060444/checkpoint-1000/infer_result/20240930-081911.jsonl (deflated 31%)
  adding: kaggle/working/hindi_got_model_3/got-ocr2/v0-20240930-060444/checkpoint-1000/additional_config.json (deflated 27%)
  adding: kaggle/working/hindi_got_model_3/got-ocr2/v0-20240930-060444/checkpoint-1000/configuration.json (deflated 40%)
  adding: kaggle/working/hindi_got_model_3/got-ocr2/v0-20240930-060444/checkpoint-1000/trainer_state.json (deflated 81%)
  adding: kaggle/working/hindi_got_model_